In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:
data = pd.read_csv("../input/chocolate-ratings/chocolate_ratings.csv")

In [4]:
data

# Verificação

In [5]:
data.isnull().sum()

In [6]:
data.dtypes

In [7]:
def RemoverPercentual(data):
    return data.apply(lambda x: float(x.strip('%'))/100)

In [8]:
data['Cocoa Percent'] = RemoverPercentual(data['Cocoa Percent'])

In [9]:
data

In [10]:
data.dtypes

In [11]:
data.drop(['REF','Review Date','Specific Bean Origin or Bar Name','Most Memorable Characteristics','Company (Manufacturer)','Company Location'], axis=1, inplace=True)

In [12]:
data

In [13]:
categorical_axis = ['Country of Bean Origin','Ingredients']

In [14]:
def Matriz_Chocolate(data, columns):
    for column in columns:
        dummies = pd.get_dummies(data[column])
        data = pd.concat([data, dummies], axis=1)
        data.drop(column, axis=1, inplace=True)
    return data    

In [15]:
data = Matriz_Chocolate(data, categorical_axis)

In [16]:
data

In [17]:
y = data['Rating']
x = data.drop('Rating', axis=1)

In [18]:
y

In [19]:
x

In [20]:
scaler = MinMaxScaler ()

x = pd.DataFrame(scaler.fit_transform(x),columns=x.columns)

In [21]:
x

# Treinamento

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.5, test_size=0.5)

In [23]:
inputs = tf.keras.Input(shape=(83,))
x = tf.keras.layers.Dense (8, activation='relu')(inputs)
x = tf.keras.layers.Dense (8, activation='relu')(x)
outputs = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [24]:
optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(
    optimizer=optimizer,
    loss='mse'
)

In [25]:
model.summary()

In [26]:
epochs=37

history = model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=epochs,
    batch_size=30,
    shuffle = True,
    verbose=2
)

In [27]:
history.history

In [28]:
plt.figure(figsize=(14,10))

plt.plot(range(epochs),history.history['loss'], color='b')
plt.plot(range(epochs),history.history['val_loss'], color='r')

plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.show()

In [29]:
np.argmin(history.history['val_loss'])

In [30]:
model.evaluate(x_test, y_test)

# Predição

In [31]:
prediction = model.predict(
    x_test,
    batch_size=30,
    verbose=0
)

In [32]:
prediction

In [33]:
y_test

In [34]:
model.get_weights()

In [35]:
model.save('models/modelo.h5')